# TGWeb Mobile Android Build (Google Colab)\nПайплайн разделен на этапы получения данных, а сборка выполняется одним отдельным шагом.

In [ ]:
%%bash\nset -euxo pipefail\nexport DEBIAN_FRONTEND=noninteractive\napt-get update\napt-get install -y --no-install-recommends ca-certificates git curl unzip zip openjdk-17-jdk-headless\n

In [ ]:
# STEP 2: upload SSH private key (id_ed25519)\nfrom google.colab import files\nuploaded = files.upload()\nassert 'id_ed25519' in uploaded, 'Upload file named id_ed25519'\n

In [ ]:
%%bash\nset -euxo pipefail\nmkdir -p /root/.ssh\nmv -f /content/id_ed25519 /root/.ssh/id_ed25519\nchmod 600 /root/.ssh/id_ed25519\nssh-keyscan github.com >> /root/.ssh/known_hosts\ngit config --global user.name 'colab-builder'\ngit config --global user.email 'colab-builder@local'\n

In [ ]:
%%bash\nset -euxo pipefail\ncd /content\nrm -rf tgwebmobile\ngit clone git@github.com:Perdonus/tgwebmobile.git\n

In [ ]:
%%bash\nset -euxo pipefail\nexport ANDROID_SDK_ROOT=/content/android-sdk\nmkdir -p "$ANDROID_SDK_ROOT/cmdline-tools"\ncurl -fL --retry 5 --retry-delay 5 https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip -o /tmp/cmdline-tools.zip\nunzip -q /tmp/cmdline-tools.zip -d "$ANDROID_SDK_ROOT/cmdline-tools"\nrm -rf "$ANDROID_SDK_ROOT/cmdline-tools/latest"\nmv "$ANDROID_SDK_ROOT/cmdline-tools/cmdline-tools" "$ANDROID_SDK_ROOT/cmdline-tools/latest"\n

In [ ]:
%%bash\nset -euxo pipefail\nexport ANDROID_SDK_ROOT=/content/android-sdk\nyes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT" --licenses\nyes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT" \\n  'platform-tools' 'platforms;android-34' 'build-tools;34.0.0'\ncat > /content/tgweb_env.sh <<'ENVEOF'\nexport JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64\nexport ANDROID_SDK_ROOT=/content/android-sdk\nexport ANDROID_HOME=$ANDROID_SDK_ROOT\nexport PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH\nENVEOF\n

In [ ]:
# STEP 8: upload app secrets/files\n# Required (if used): google-services.json, keystore.jks, keystore.properties\nfrom google.colab import files\nfiles.upload()\n

In [ ]:
%%bash\nset -euxo pipefail\ncd /content/tgwebmobile\nsource /content/tgweb_env.sh\ncat > local.properties <<EOF2\nsdk.dir=/content/android-sdk\nEOF2\nif [ -f /content/google-services.json ]; then cp /content/google-services.json app/google-services.json; fi\nif [ -f /content/keystore.jks ]; then cp /content/keystore.jks app/keystore.jks; fi\nif [ -f /content/keystore.properties ]; then cp /content/keystore.properties keystore.properties; fi\n

In [ ]:
%%bash\nset -euxo pipefail\nsource /content/tgweb_env.sh\ncd /content/tgwebmobile\n./gradlew --no-daemon :backend:push:build :app:assembleDebug\n